In [1]:
import itk
from itk import ThinShellDemonsMetricv4 as tsd
from itk import ConjugateGradientLineSearchOptimizerv4Template as itkc

import numpy as np
import math
import itkwidgets
from itkwidgets import view

In [2]:
# For TSD Registration

basepath = '/media/pranjal.sahu/moredata/ITK/Modules/External/ITKThinShellDemons/test/Baseline/'
fixedMesh = itk.meshread(basepath + 'fixedMesh.vtk', itk.D)
movingMesh = itk.meshread(basepath + 'movingMesh.vtk', itk.D)

fixedMesh.BuildCellLinks()
movingMesh.BuildCellLinks()

PixelType = itk.D
Dimension = 3

MeshType = itk.Mesh[itk.D, Dimension]
FixedImageType = itk.Image[PixelType, Dimension]
MovingImageType = itk.Image[PixelType, Dimension]


# For getting the Bounding Box
ElementIdentifierType = itk.UL
CoordType = itk.F
Dimension = 3

VecContType = itk.VectorContainer[
    ElementIdentifierType, itk.Point[CoordType, Dimension]
]
bounding_box = itk.BoundingBox[ElementIdentifierType, Dimension, CoordType, VecContType].New()
bounding_box.SetPoints(movingMesh.GetPoints())
bounding_box.ComputeBoundingBox()

minBounds = np.array(bounding_box.GetMinimum())
maxBounds = np.array(bounding_box.GetMaximum())

imageDiagonal = 5
spacing = np.sqrt(bounding_box.GetDiagonalLength2()) / imageDiagonal
diff = maxBounds - minBounds

fixedImageSize = [0]*3
fixedImageSize[0] = math.ceil( 1.2 * diff[0] / spacing )
fixedImageSize[1] = math.ceil( 1.2 * diff[1] / spacing )
fixedImageSize[2] = math.ceil( 1.2 * diff[2] / spacing )

fixedImageOrigin = [0]*3
fixedImageOrigin[0] = minBounds[0] - 0.1 * diff[0]
fixedImageOrigin[1] = minBounds[1] - 0.1 * diff[1]
fixedImageOrigin[2] = minBounds[2] - 0.1 * diff[2]

fixedImageSpacing = np.ones(3)*spacing
fixedImageDirection = np.identity(3)


fixedImage = FixedImageType.New()
fixedImage.SetRegions(fixedImageSize)
fixedImage.SetOrigin( fixedImageOrigin )
fixedImage.SetDirection( fixedImageDirection )
fixedImage.SetSpacing( fixedImageSpacing )
fixedImage.Allocate()


# Affine Transform Object
TransformType = itk.AffineTransform.D3
transform = TransformType.New()
transform.SetIdentity()
transform.SetCenter(minBounds + (maxBounds - minBounds)/2)

print('Transform Created')
print(transform)


MetricType = tsd.MD3
metric = MetricType.New()
metric.SetStretchWeight(1)
metric.SetBendWeight(5)
metric.SetGeometricFeatureWeight(10)
metric.UseConfidenceWeightingOn()
metric.UseMaximalDistanceConfidenceSigmaOn()
metric.UpdateFeatureMatchingAtEachIterationOff()
metric.SetMovingTransform(transform)
# Reversed due to using points instead of an image
# to keep semantics the same as in itkThinShellDemonsTest.cxx
# For the ThinShellDemonsMetricv4 the fixed mesh is regularized
metric.SetFixedPointSet(movingMesh)
metric.SetMovingPointSet(fixedMesh)
metric.SetVirtualDomainFromImage(fixedImage)
metric.Initialize()

print('TSD Metric Created')

shiftScaleEstimator = itk.RegistrationParameterScalesFromPhysicalShift[MetricType].New()
shiftScaleEstimator.SetMetric(metric)
shiftScaleEstimator.SetVirtualDomainPointSet(metric.GetVirtualTransformedPointSet())


optimizer = itkc.D.New()
optimizer.SetNumberOfIterations( 50 )
optimizer.SetScalesEstimator( shiftScaleEstimator )
optimizer.SetMaximumStepSizeInPhysicalUnits( 0.5 )
optimizer.SetMinimumConvergenceValue( 0.0 )
optimizer.SetConvergenceWindowSize( 10 )

def iteration_update():
    metric_value = optimizer.GetValue()
    current_parameters = optimizer.GetCurrentPosition()
    print(f"Metric: {metric_value:.8g}")

iteration_command = itk.PyCommand.New()
iteration_command.SetCommandCallable(iteration_update)
optimizer.AddObserver(itk.IterationEvent(), iteration_command)

print('Optimizer created')


AffineRegistrationType = itk.ImageRegistrationMethodv4.REGv4D3D3TD3D3MD3.New()
registration = AffineRegistrationType.New()
registration.SetNumberOfLevels(1)
registration.SetObjectName("registration")
registration.SetFixedPointSet(movingMesh)
registration.SetMovingPointSet(fixedMesh)
registration.SetInitialTransform(transform)
registration.SetMetric(metric)
registration.SetOptimizer(optimizer)

print('Registration Object created')
print('Initial Value of Metric ', metric.GetValue())

try:
    registration.Update()
except e:
    print('Error is ', e)

print('Final Value of Metric ', metric.GetValue())

finalTransform = registration.GetModifiableTransform()
numberOfPoints = movingMesh.GetNumberOfPoints()
for n in range(0, numberOfPoints):
    movingMesh.SetPoint(n, finalTransform.TransformPoint(movingMesh.GetPoint(n)))

Transform Created
AffineTransform (0x55d3d588b0e0)
  RTTI typeinfo:   itk::AffineTransform<double, 3u>
  Reference Count: 1
  Modified Time: 10306
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    1 0 0 
    0 1 0 
    0 0 1 
  Offset: [0, 0, 0]
  Center: [-3.68575, 637.172, 463.569]
  Translation: [0, 0, 0]
  Inverse: 
    1 0 0 
    0 1 0 
    0 0 1 
  Singular: 0

TSD Metric Created
Optimizer created
Registration Object created
Initial Value of Metric  9.303174789139529
Metric: 9.3031748
Metric: 8.4354592
Metric: 7.8122719
Metric: 7.5588837
Metric: 7.5430234
Metric: 7.4710142
Metric: 7.4233074
Metric: 7.4129552
Metric: 7.408688
Metric: 7.3804732
Metric: 7.3775668
Metric: 7.3540113
Metric: 7.2890243
Metric: 7.2563025
Metric: 7.2430692
Metric: 7.1849164
Metric: 7.1357237
Metric: 7.1087634
Metric: 7.1087659
Metric: 7.1087637
Metric: 7.1087639
Metric: 7.1087633
Metric: 7.1087633
Metric: 7.108762
Metric: 7.108989
Metric: 7.10899
Final Value of Metric  7.11070797955915


In [3]:
itk.meshwrite(movingMesh, "affineMovingMesh.vtk")

In [1]:
moving_mesh = itk.meshread('/media/pranjal.sahu/moredata/ITKPR30/ITK/ITK-build/Wrapping/Generators/Python/movingMesh_transformed.vtk')

NameError: name 'itk' is not defined

In [11]:
# For converting from PLY to VTK

import vtk

reader = vtk.vtkPLYReader()
reader.SetFileName('/home/pranjal.sahu/Downloads/129X1_SVJ_.ply')
reader.Update()
polydata1 = reader.GetOutput()

reader1 = vtk.vtkPLYReader()
reader1.SetFileName('/home/pranjal.sahu/Downloads/129S1_SVIMJ_.ply')
reader1.Update()
polydata2 = reader1.GetOutput()

print(polydata1.GetNumberOfPoints())
print(polydata2.GetNumberOfPoints())


writer = vtk.vtkPolyDataWriter()
writer.SetFileName("/home/pranjal.sahu/Downloads/129X1_SVJ_.vtk")
writer.SetFileVersion(42)
writer.SetInputData(polydata1)
writer.Update()

writer = vtk.vtkPolyDataWriter()
writer.SetFileName("/home/pranjal.sahu/Downloads/129S1_SVIMJ_.vtk")
writer.SetFileVersion(42)
writer.SetInputData(polydata2)
writer.Update()

169954
174916


In [16]:
import vtk

reader = vtk.vtkPolyDataReader()
reader.SetFileName('/media/pranjal.sahu/moredata/ITKPR30/ITK/ITK-build/Wrapping/Generators/Python/movingMesh_transformed.vtk')
reader.Update()
polydata1 = reader.GetOutput()

reader = vtk.vtkPolyDataReader()
reader.SetFileName('/home/pranjal.sahu/Downloads/129X1_SVJ_.vtk')
reader.Update()
polydata2 = reader.GetOutput()

reader = vtk.vtkPolyDataReader()
reader.SetFileName('/home/pranjal.sahu/Downloads/129S1_SVIMJ_.vtk')
reader.Update()
polydata3 = reader.GetOutput()

In [17]:
import itkwidgets
from itkwidgets import view

#view(geometries=[polydata1, polydata2, polydata3])
view(geometries=[polydata1,  polydata3])
#view(geometries=[polydata])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [8]:
# For ICP registration

Dimension = 3

PointSetType = itk.PointSet[itk.F, Dimension]

fixedMesh = itk.meshread('/home/pranjal.sahu/Downloads/129X1_SVJ_.vtk')
movingMesh = itk.meshread('/home/pranjal.sahu/Downloads/129S1_SVIMJ_.vtk')

fixedPS = PointSetType.New()
movingPS = PointSetType.New()

fixedPS.SetPoints(fixedMesh.GetPoints())
movingPS.SetPoints(movingMesh.GetPoints())

print(fixedMesh.GetNumberOfPoints(), movingMesh.GetNumberOfPoints())
print(fixedPS.GetNumberOfPoints(), movingPS.GetNumberOfPoints())

169954 174916
169954 174916


In [35]:
OptimizerType    = itk.LevenbergMarquardtOptimizer
MetricType       = itk.EuclideanDistancePointSetToPointSetMetricv4[type(fixedPS)]
#MetricType       = itk.EuclideanDistanceMetricv[type(fixedPS)]
TransformType    = itk.Rigid3DTransform[itk.D]
RegistrationType = itk.PointSetToPointSetRegistrationMethod[PointSetType, PointSetType]

#print(MetricType)

In [23]:
metric       = MetricType.New()
transform    = TransformType.New()
optimizer    = OptimizerType.New()
registration = RegistrationType.New()

In [24]:
numberOfIterations = 100
gradientTolerance  = 1e-5
valueTolerance     = 1e-5   
epsilonFunction    = 1e-6

#optimizer.SetScales(scales)
optimizer.SetNumberOfIterations(numberOfIterations)
optimizer.SetValueTolerance(valueTolerance)
optimizer.SetGradientTolerance(gradientTolerance)
optimizer.SetEpsilonFunction(epsilonFunction)

In [31]:
metric.SetFixedPointSet(fixedPS)
metric.SetMovingPointSet(movingPS)
metric.SetMovingTransform(TransformType)
metric.Initialize()

TypeError: in method 'itkObjectToObjectMetric33_SetMovingTransform', argument 2 of type 'itkTransformD33 *'

In [33]:
AffineRegistrationType = itk.ImageRegistrationMethodv4.REGv4D3D3TD3D3MD3.New()
registration = AffineRegistrationType.New()
registration.SetNumberOfLevels(1)
registration.SetObjectName("registration")
registration.SetFixedPointSet(movingMesh)
registration.SetMovingPointSet(fixedMesh)
registration.SetInitialTransform(transform)
registration.SetMetric(metric)
registration.SetOptimizer(optimizer)

itk.itkRigid3DTransformPython.itkRigid3DTransformD

In [25]:
def iteration_update():
    metric_value = optimizer.GetValue()
    current_parameters = optimizer.GetCurrentPosition()
    print(f"Metric: {metric_value:.8g}")

iteration_command = itk.PyCommand.New()
iteration_command.SetCommandCallable(iteration_update)
optimizer.AddObserver(itk.IterationEvent(), iteration_command)


transform.SetIdentity()
registration.SetInitialTransformParameters(transform.GetParameters())
registration.SetMetric(metric)
registration.SetOptimizer(optimizer)
registration.SetTransform(transform)
registration.SetFixedPointSet(fixedPointSet)
registration.SetMovingPointSet(movingPointSet)
registration.Update()

TypeError: in method 'itkPointSetToPointSetRegistrationMethodREGF3F3_SetMetric', argument 2 of type 'itkPointSetToPointSetMetricPSF3 *'

In [28]:
registration.SetMetric(metric)

TypeError: in method 'itkPointSetToPointSetRegistrationMethodREGF3F3_SetMetric', argument 2 of type 'itkPointSetToPointSetMetricPSF3 *'

In [29]:
type(metric)

itk.itkEuclideanDistancePointSetToPointSetMetricPython.itkEuclideanDistancePointSetToPointSetMetricv4PSF3

In [30]:
type(registration)

itk.itkPointSetToPointSetRegistrationMethodPython.itkPointSetToPointSetRegistrationMethodREGF3F3